# Evidencia 2

Nombres: Diego Lira García
        Andrea Medina Rico

## Importaciones

In [11]:
%pip install mesa==2.3.1 --quiet
%pip install matplotlib numpy pandas --quiet
%pip install seaborn --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que puedan existir varios agentes en una celda, elegimos ''MultiGrid''.
from mesa.space import MultiGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# Librería para modificar parámetros --> Corre varias simulaciones
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random
import heapq

## Lectura de archivos

In [13]:
def read_file(filename):
    """ Lee el archivo de texto y procesa las matrices requeridas. """
    # Leer el archivo
    with open(filename, 'r') as archivo:
        lines = archivo.readlines()
    
    # Leer primeras 6 líneas --> Paredes
    wall_cells = read_matrix_from_file(lines, 0, 6)

    # Leer siguientes 3 líneas --> Points of interest
    # Sí se requiere crear la CLASE
    poi_cells = save_as_tuple_str(lines, 6, 9)

    # Leer siguientes 10 líneas --> Fire
    fire_cells = save_as_tuple(lines, 9, 19)

    # Leer siguientes 8 líneas --> Doors
    door_cells = save_as_tuple(lines, 19, 27)

    # Leer siguientes 4 líneas --> últimas 4 líneas --> Outside doors
    outside_doors = save_as_tuple(lines, 27, 31)
    
    # Procesamiento de matrices
    doors_matriz = create_doors_matrix(door_cells, outside_doors)
    walls_matriz = create_walls_matrix(wall_cells, doors_matriz)
    fire_matriz = create_fire_matrix(fire_cells)
    poi_matriz, poi_cells_initial = create_poi_matrix(poi_cells)

    return walls_matriz, doors_matriz, fire_matriz, poi_matriz, poi_cells_initial, outside_doors


def save_as_tuple(lines, start, end):
    """ Guarda las líneas del archivo como tuplas. """
    cells = []
    for i in range(start, end):
        row = tuple(map(int, lines[i].strip().split()))
        cells.append(row)
    print(cells)
    return cells


def save_as_tuple_str(lines, start, end):
    """ Guarda las líneas del archivo como tuplas con strings y enteros. """
    cells = []
    for i in range(start, end):
        parts = lines[i].strip().split()
        row = (int(parts[0]), int(parts[1]), parts[2])
        cells.append(row)
    return cells

def read_matrix_from_file(lines, start, end):
    """
    Lee un archivo de texto donde cada celda está representada por cuatro números,
    y convierte los datos en una matriz con el formato especificado.
    """
    matriz = []
    for line in lines[start:end]:
        # Divide la línea en grupos de cuatro caracteres
        fila = [list(map(int, cell)) for cell in line.strip().split()]
        matriz.append(fila)
    print(matriz)
    return matriz


def create_doors_matrix(door_cells, outside_doors):
    """ Crea matriz de puertas externas e internas en ambas celdas. """
    matriz = [[[0, 0, 0, 0] for _ in range(10)] for _ in range(8)]

    # Agregar puertas del interior
    for connection in door_cells:
        x1, y1, x2, y2 = connection

        if 0 <= x1 <= 6 and 0 <= y1 <= 8 and 0 <= x2 <= 6 and 0 <= y2 <= 8:
            if x1 == x2:
                if y2 == y1 + 1:
                    matriz[x1][y1][3] = 2
                    matriz[x2][y2][1] = 2
                elif y2 == y1 - 1:
                    matriz[x1][y1][1] = 2
                    matriz[x2][y2][3] = 2
            elif y1 == y2:
                if x2 == x1 + 1:
                    matriz[x1][y1][2] = 2
                    matriz[x2][y2][0] = 2
                elif x2 == x1 - 1:
                    matriz[x1][y1][0] = 2
                    matriz[x2][y2][2] = 2
        else:
            print(f"Index out of range for interior doors: {connection}")

    # Agregar puertas del exterior
    for position in outside_doors:
        x, y = position

        if 0 <= x <= 6 and 0 <= y <= 8:
            if x == 1:
                matriz[x][y][0] = 2
                matriz[x - 1][y][2] = 2
            elif x == 6:
                matriz[x][y][2] = 2
                matriz[x + 1][y][0] = 2
            elif y == 1:
                matriz[x][y][1] = 2
                matriz[x][y - 1][3] = 2
            elif y == 8:
                matriz[x][y][3] = 2
                matriz[x][y + 1][1] = 2
        else:
            print(f"Index out of range for exterior doors: {position}")

    return matriz


def create_walls_matrix(wall_cells, doors_matriz):
    """ Crea matriz de paredes en ambas celdas considerando puertas y celdas externas. """

    # Crear matriz extendida
    outside_walls = [[[0, 0, 0, 0] for _ in range(10)] for _ in range(8)]

    # Copia de celdas de paredes internas
    for i in range(6):
        for j in range(8):
            outside_walls[i + 1][j + 1] = wall_cells[i][j]
    
    # Ajuste de paredes externas
    for x in range(1, 7):
        # Primera columna: pared derecha
        outside_walls[x][0] = [0, 0, 0, 1]
        # Última columna: pared izquierda
        outside_walls[x][9] = [0, 1, 0, 0]

    for y in range(1, 9):
        # Primera fila: pared abajo
        outside_walls[0][y] = [0, 0, 1, 0]
        # Última fila: pared arriba
        outside_walls[7][y] = [1, 0, 0, 0]

    # Quitar pared donde hay puerta
    for x in range(8):
        for y in range(10):
            for i in range(4):
                if doors_matriz[x][y][i] == 1 or doors_matriz[x][y][i] == 2:
                    outside_walls[x][y][i] = 0
    
    return outside_walls


def create_fire_matrix(fire_cells):
    """ Crea matriz posicionando el fuego en las celdas designadas. """
    fire_matriz = np.zeros((8, 10))

    for x, y in fire_cells:
        if 0 <= x < 8 and 0 <= y < 10:
            fire_matriz[x][y] = 2
        else:
            print(f"Index out of range for fire: {x, y}")

    return fire_matriz


def create_poi_matrix(poi_cells):
    """ Crea matriz posicionando los puntos de interés en las celdas designadas. """
    poi_matriz = np.zeros((8, 10))

    index = 0
    for x, y, v in poi_cells:
        if 0 <= x < 8 and 0 <= y < 10:
            # Posiciona POI en la matriz
            poi_matriz[x][y] = 1

            # Cambia valor de víctima o alarma por su número
            if v == "v":
                poi_cells[index] = (x, y, 2)
            elif v == "f":
                poi_cells[index] = (x, y, 3)
            index += 1

        else:
            print(f"Index out of range for poi: {x, y}")

    return poi_matriz, poi_cells

## 1. Solución aleatoria

### Funciones globales auxiliares

In [14]:
def get_contrapart_index(index):
    """ Obtiene qué indice de la celda es la contraparte del índice actual. 
    Es decir, obtiene el índice contrario de donde se encuentra la pared/puerta. """
    # Archivo:  0 arriba, 1 izquierda, 2 abajo, 3 derecha
    if index == 0:
        return 2
    elif index == 1:
        return 3
    elif index == 2:
        return 0
    elif index == 3:
        return 1
    

def get_contrapart_cell(x, y, index):
    """ Obtiene la celda contraparte de la celda actual. 
    Es decir, obtiene la celda contraria de donde se encuentra la pared/puerta
    de acuerdo al índice. """
    
    if index == 0:  
        return x - 1, y
    elif index == 1:
        return x, y - 1
    elif index == 2:
        return x + 1, y
    elif index == 3:
        return x, y + 1
    

def get_contrapart_cell_index(x_curr, y_curr, x_next, y_next):
    """ Obtiene de qué lado se encuentra la celda contraparte de la celda actual
    y regresa el índice indicado que se tiene que checar desde la celda actual. """

    if x_curr == x_next:
        if y_curr < y_next:     # Derecha
            return 3
        elif y_curr > y_next:   # Izquierda
            return 1
    elif y_curr == y_next:
        if x_curr < x_next:     # Abajo
            return 2
        elif x_curr > x_next:   # Arriba
            return 0
    

def random_number(min, max):
    """ Regresa un número aleatorio dentro del límite. """
    return np.random.randint(min, max + 1)


def create_border_matrix(height, width):
    """Crea una matriz donde las orillas son 0 y el interior es 1"""
    matrix = []
    for i in range(height):
        row = []
        for j in range(width):
            if i == 0 or i == height - 1 or j == 0 or j == width - 1:
                row.append(0)  # Orillas
            else:
                row.append(1)  # Interior
        matrix.append(row)
    return matrix


### Implementación del agente

In [15]:
class FirefighterAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

        self.action_points = 4
        self.carrying_victim = False
        self.knocked_down = False
        self.points_saved = 0
        self.action_made = None

    
    def step(self):
        self.action_made = None

        if self.action_points > 0:
            print("Puntos de acción", self.action_points)
            while self.action_made is None:
                # Asignar un número a cada función y que elija aleatoriamente
                option = random_number(0, 5)

                if option == 0:
                    self.move_to_cell()

                elif option == 1:
                    self.toggle_door(self.pos)

                elif option == 2:
                    self.extinguish(self.pos, random_number(1, 3))

                elif option == 3:
                    self.chop()

                elif option == 4:
                    self.toggle_victim()

                elif option == 5 and self.action_points <= 4:
                    self.save_points()
            print("Acción realizada", self.action_made, "\n")

        # Fin de turno
        elif self.action_points == 0:
            self.end_turn()
        


    def move_to_cell(self):
        """ Mueve al agente a una celda adyacente que no esté bloqueada por una puerta o pared. """
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)
        indexes = [x for x in range( len(possible_positions) )]
        np.random.shuffle(indexes)
        print("Posición actual", self.pos)
        print("Posiciones posibles", possible_positions)
        print("Índices", indexes)

        for i in indexes:
            position = possible_positions[i]
            x, y = self.pos
            status = self._get_cell_status(position)
            index = get_contrapart_cell_index(x, y, position[0], position[1])

            print("Checa su lado", index)
            if self.model.walls[x][y][index] == 0 and self.model.doors[x][y][index] != 2:
                self.action_made = "move"
                
                print("Posición a la que se moverá", position)

                if status == 2 and not self.carrying_victim and self.action_points >= 2:
                    self.model.grid.move_agent(self, position)
                    self.action_points -= 2
                elif self.carrying_victim and status != 2 and self.action_points >= 2:
                    self.action_points -= 2
                    self.model.grid.move_agent(self, position)
                elif self.action_points >= 1:
                    self.action_points -= 1
                    self.model.grid.move_agent(self, position)
                else:
                    self.action_made = None
                    continue

                self._check_for_poi(self.pos)
                # Si salió y carga un agente
                self._save_victim()

                break

  
    def toggle_door(self, position):
        """ Cambia el estado de la puerta en la celda (x, y). """
        x, y = position
        cell = self.model.doors[x][y]

        # Obtener cuatro posible celdas  --> 0 Arriba, 1 derecha, 2 abajo, 3 izquierda
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)

        # Encontrar la puerta que se quiere alterar
        # 0 Arriba, 1 izquierda, 2 abajo, 3 derecha
        # 1 Abierto 2 Cerrado
        if self.action_points >= 1:
            index = 0
            for i in cell:
                if i != 0:
                    if i == 1:
                        cell[index] = 2
                        new_state = 2
                    else:
                        cell[index] = 1
                        new_state = 1
                    print("Puerta encontrada de lado", index)
                    print("Puerta encontrada", i, "pasa a estado", new_state)
                    print("Puerta pasada a", new_state, "en la celda", self.pos, "de lado", index)

                    self.action_points -= 1
                    self.action_made = "toggle_door"
                    break
                index += 1
            
            if self.action_made == "toggle_door":
                # Índice opuesto
                other_index = get_contrapart_index(index)
                x_other, y_other = get_contrapart_cell(x, y, index)
                
                # Cambiar el estado de la otra celda
                next_cell = self.model.doors[x_other][y_other]
                next_cell[other_index] = new_state
                print("Puerta opuesta encontrada de lado", other_index)
                print("Puerta pasada a", new_state, "en la celda", x_other, y_other, "de lado", other_index)
                return
    
    
    def extinguish(self, position, option):
        """ Extingue el fuego o el humo en la celda (x, y). """
        x, y = position

        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)
        indexes = [x for x in range( len(possible_positions) )]
        np.random.shuffle(indexes)

        for i in indexes:
            position = possible_positions[i]
            x, y = position
            x_self, y_self = self.pos
            cell = self._get_cell_status(position)
            index = get_contrapart_cell_index(x_self, y_self, x, y)

            self.action_made = "extinguish"

            if self.model.walls[x_self][y_self][index] == 0 and self.model.doors[x_self][y_self][index] != 2:

                if option == 1 and cell == 2 and self.action_points >= 2:
                    self.model.grid_status[x][y] = 0
                    self.action_points -= 2
                    return
                elif option == 2 and cell == 2 and self.action_points >= 1:
                    self.model.grid_status[x][y] = 1
                    self.action_points -= 1
                    return
                elif option == 3 and cell == 1 and self.action_points >= 1:
                    self.model.grid_status[x][y] = 0
                    self.action_points -= 1
                    return
                else:
                    self.action_made = None
                    continue
        
        x, y = self.pos

        # Apagar fuego de la celda actual
        if option == 1 and cell == 2 and self.action_points >= 2:
            self.model.grid_status[x][y] = 0
            self.action_points -= 2
        elif option == 2 and cell == 2 and self.action_points >= 1:
            self.model.grid_status[x][y] = 1
            self.action_points -= 1
        elif option == 3 and cell == 1 and self.action_points >= 1:
            self.model.grid_status[x][y] = 0
            self.action_points -= 1
        else:
            self.action_made = None

            

    def chop(self):
        """ Daña la pared en la celda (x, y). """
        xs, ys = self.pos
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)

        for x, y in possible_positions:
            index_curr_to_adj = get_contrapart_cell_index(xs, ys, x, y)
            other_adj_to_curr = get_contrapart_index(index_curr_to_adj)

            wall = self.model.walls[xs][ys][index_curr_to_adj]
            print("Pared encontrada", wall, "de lado", index_curr_to_adj, "en celda", xs, ys)
            
            if wall > 0 and self.action_points >= 2:
                self.model.walls[x][y][other_adj_to_curr] -= 0.5
                print("Pared dañada de lado", other_adj_to_curr, "en celda", x, y)
                self.model.walls[xs][ys][index_curr_to_adj] -= 0.5
                print("Pared dañada de lado", index_curr_to_adj, "en celda", xs, ys)
                self.action_points -= 2
                self.model.damage_counters -= 1
                self.action_made = "chop"



    def toggle_victim(self):
        """ Toma o deja a la víctima en la celda (x, y). """
        x, y = self.pos
        cell = self.model.poi[x][y]

        if cell == 2 and not self.carrying_victim:
            self.carrying_victim = True
            self.model.poi[x][y] = 0
            self.action_made = "carry_victim"

        elif cell == 0 and self.carrying_victim:
            self.carrying_victim = False
            self.model.poi[x][y] = 2
            self.action_made = "leave_victim"


    def save_points(self):
        self.points_saved = self.action_points
        self.action_made = "save_points"
        self.end_turn()


    def end_turn(self):
        """ Finaliza turno del agente, pasando a la siguiente fase de juego y al siguiente jugador. """
        self.action_points = 4 + self.points_saved
        self.points_saved = 0
        if self.action_points > 8:
            self.action_points = 8

        self.model.game_phase = "Advance Fire"


        if self.model.active_player < 5:
            self.model.active_player += 1
        else:
            self.model.active_player = 0


    # Métodos auxiliares
    def _check_for_poi(self, position):
        """ Verifica si la celda (x, y) tiene un punto de interés y qué tipo de punto es. """
        x, y = position
        cell = self.model.poi[x][y]

        if cell == 1:
            choice = self.model._choose_between_victim_or_false_alarm(x, y)
            self.model.poi[x][y] = choice


    def _save_victim(self):
        """ Deja a la víctima en una celda exterior. """
        if self.carrying_victim and self.pos in self.model.edge_positions:
            self.carrying_victim = False
            self.model.victims_saved += 1
        
        
    def _get_cell_status(self, position):
        """ Obtiene el contenido de una celda (humo o fuego). """
        x, y = position
        return self.model.grid_status[x][y]
    

### Implementación de modelo

In [16]:
class FlashPointModel(Model):

    def __init__(self, walls_matrix, doors_matrix, fire_matrix, poi_matrix, poi_cells, outside_matrix, 
                 width = 10, height = 8, agents = 6, damage_counters = 24):

        super().__init__()
        self.grid = MultiGrid(height, width, torus = False)
        self.schedule = SimultaneousActivation(self)

        self.datacollector = DataCollector()

        # Matrices
        self.walls = walls_matrix
        self.doors = doors_matrix
        self.outside_doors = outside_matrix
        self.grid_status = fire_matrix
        self.poi = poi_matrix
        self.poi_cells = poi_cells

        self.saved_victims = 0
        self.lost_victims = 0
        self.damage_counters = damage_counters
        self.false_alarms = 6
        self.total_victims = 12
        self.point_of_interest = 3
        self.steps = 0
        self.width = width
        self.height = height
        self.game_phase = "Player"
        self.agents_by_id = {}
        self.active_player = 0
        self.victory = False
        self.defeat = False

        self.edge_positions = self.get_edge_positions(width, height)

        # Creación agentes
        for i in range(agents):
            agent = FirefighterAgent(i, self)
            self.schedule.add(agent)
            self.agents_by_id[i] = agent

            # Posicionar
            agent_placed = False
            while not agent_placed:
                chosen_pos = random.choice(self.edge_positions)
                if self.grid.is_cell_empty(chosen_pos):
                    self.grid.place_agent(agent, chosen_pos)
                    agent_placed = True


    def get_edge_positions(self, width, height):
        """ Obtiene las celdas de los bordes del tablero. """
        edge_positions = []
        for x in range(0, height - 1):
            edge_positions.append((x, 0))  # Orilla superior
            edge_positions.append((x, 9))  # Orilla inferior

        for y in range(0, width - 1):
            edge_positions.append((0, y))  # Orilla izquierda
            edge_positions.append((7, y))  # Orilla derecha

        return edge_positions
    

    def advance_fire(self):
        """ Coloca un humo aletorio. """
        x, y = self.roll_dice()
        cell = self.grid_status[x][y]
        
        # Si la celda está vacía, se coloca humo
        if cell == 0:
            self.grid_status[x][y] = 1
        elif cell == 1:
            self.grid_status[x][y] = 2
        elif cell == 2:
            self.explosion(x, y)
        
        self.game_phase = "Secondary Effects"


    def explosion(self, x, y):
        """ Realiza una explosión y llama a explosion_effect para las celdas adyacentes. """
        print("Dentro de explosion en la celda", x, y)
        adjacent_cells = self.grid.get_neighborhood((x, y), moore = False, include_center = False)
        
        index = 0
        for position in adjacent_cells:
            self.explosion_effect(position[0], position[1], index, x, y)
            index += 1
        
        
    def explosion_effect(self, adj_x, adj_y, index, current_x, current_y):
        """ Verifica qué encuentra en las celdas adyacentes y genera el efecto correspondiente. """
        # Index 0 arriba, 1 izquierda, 2 derecha, 3 abajo
        cell = self.grid_status[adj_x][adj_y]
        index_curr_to_adj = get_contrapart_cell_index(current_x, current_y, adj_x, adj_y)
        index_adj_to_curr = get_contrapart_index(index_curr_to_adj)
        door = self.doors[current_x][current_y][index_curr_to_adj]
        wall = self.walls[current_x][current_y][index_curr_to_adj]
        
        # Puerta cerrada se destruye
        if door == 2:
            self.doors[adj_x][adj_y][index_adj_to_curr] = 0
            self.doors[current_x][current_y][index_curr_to_adj] = 0  
            print("Puerta destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
            print("Puerta destruida en la celda", adj_x, adj_y, "de lado", index_adj_to_curr)

        # Puerta abierta se destruye y avanza fuego
        elif door == 1:
            self.doors[adj_x][adj_y][index_adj_to_curr] = 0
            self.doors[current_x][current_y][index_curr_to_adj] = 0
            self.grid_status[adj_x][adj_y] = 2
            print("Puerta destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
            print("Puerta destruida en la celda", adj_x, adj_y, "de lado", index_adj_to_curr)

        # Pared se daña
        elif wall == 1 or wall == 0.5:
            self.walls[adj_x][adj_y][index_adj_to_curr] -= 0.5
            self.walls[current_x][current_y][index_curr_to_adj] -= 0.5
            self.damage_counters -= 1
            print("Pared dañada en la celda", current_x, current_y, "de lado", index_curr_to_adj)
            print("Pared dañada en la celda", adj_x, adj_y, "de lado", index_adj_to_curr)

        elif wall == 0:
            if cell == 2:
                print("Shockwave en la celda", adj_x, adj_y, "en dirección", index_curr_to_adj)
                self.shockwave(adj_x, adj_y, index_curr_to_adj)

            # Prende fuego a la siguiente celda
            else:
                self.grid_status[adj_x][adj_y] = 2
                print("Fuego en la celda", adj_x, adj_y)

                
                
    def shockwave(self, initial_x, initial_y, index_curr_to_adj):
        """ Realiza una onda expansiva de llammas en una dirección hasta no encontrar más fuego."""
        print("Dentro de shockwave")
        keep_going = True
        current_x, current_y = initial_x, initial_y

        while keep_going:
            adjacent_cells = self.grid.get_neighborhood((current_x, current_y), moore = False, include_center = False)
            # La siguiente celda es siempre en la misma dirección
            next_fire = adjacent_cells[index_curr_to_adj]
            next_x, next_y = next_fire

            # Shockwave avanza a través del fuego SI NO HAY PUERTA O PARED
            door = self.doors[current_x][current_y][index_curr_to_adj]
            wall = self.walls[current_x][current_y][index_curr_to_adj]
            index_adj_to_curr = get_contrapart_index(index_curr_to_adj)

            if self.grid_status[next_x][next_y] == 2:
                if door != 2 and wall != 1:
                    # Puerta abierta se destruye y avanza fuego
                    if door == 1:
                        self.doors[next_x][next_y][index_adj_to_curr] = 0
                        self.doors[current_x][current_y][index_curr_to_adj] = 0
                        self.grid_status[next_x][next_y] = 2
                        print("Puerta destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
                        print("Puerta destruida en la celda", next_x, next_y, "de lado", index_adj_to_curr)

                    # Pared dañada se destruye y avanza fuego
                    elif wall == 0.5:
                        self.walls[next_x][next_y][index_adj_to_curr] == 0
                        self.walls[current_x][current_y][index_curr_to_adj] == 0
                        self.grid_status[next_x][next_y] = 2
                        self.damage_counters -= 1
                        print("Pared destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
                        print("Pared destruida en la celda", next_x, next_y, "de lado", index_adj_to_curr)

                    print("Fuego avanza a la celda", next_x, next_y)
                    current_x, current_y = next_fire

            else:
                keep_going = False  
                print("Se rompe shockwave en la celda", next_x, next_y)
                
                # Puerta cerrada se destruye
                if door == 2:
                    self.doors[next_x][next_y][index_adj_to_curr] = 0
                    self.doors[current_x][current_y][index_curr_to_adj] = 0 
                    print("Puerta destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
                    print("Puerta destruida en la celda", next_x, next_y, "de lado", index_adj_to_curr) 

                # Pared se daña
                elif wall == 1:
                    self.walls[next_x][next_y][index_adj_to_curr] -= 0.5
                    self.walls[current_x][current_y][index_curr_to_adj] -= 0.5
                    self.damage_counters -= 1
                    print("Pared dañada en la celda", current_x, current_y, "de lado", index_curr_to_adj)
                    print("Pared dañada en la celda", next_x, next_y, "de lado", index_adj_to_curr)

                # Se coloca fuego continuo
                else:
                    self.grid_status[next_x][next_y] = 2
                    print("Fuego en la celda", next_x, next_y)


    def secondary_effects(self):
        """ Verifica los efectos secundarios tras la aparición de humo o fuego. """
        self.clean_fire_on_edges()
        self.flashover()
        self.knock_down()
        self.check_poi_on_fire()
        self.check_false_alarms()
        self.game_phase = "Replenish POI"
        

    def clean_fire_on_edges(self):
        """ Limpia el fuego en las celdas de los bordes. """
        for x, y in self.edge_positions:
            self.grid_status[x][y] = 0



    def knock_down(self):
        """ Verifica si un bombero es derribado por el fuego y lo coloca en la puerta más cercana. """
        for agent in self.schedule.agents:
            x, y = agent.pos
            cell = self.grid_status[x][y]
            if cell == 2:
                if agent.carrying_victim:
                    self.lost_victims += 1
                    agent.carrying_victim = False
                agent.knocked_down = True

                # Colocarlo en la puerta externa más cercana
                self.grid.move_agent(agent, self.get_closest_door(x, y))


    def get_closest_door(self, x, y):
        """ Obtiene la puerta externa más cercana a la celda (x, y). """
        min_distance = float('inf')
        closest = None

        for door in self.outside_doors:
            distance = abs(x - door[0]) + abs(y - door[1])
            if distance < min_distance:
                min_distance = distance
                closest = door

        if closest[0] == 1:
            return (closest[0] - 1, closest[1])
        elif closest[0] == 6:
            return (closest[0] + 1, closest[1])
        elif closest[1] == 1:
            return (closest[0], closest[1] - 1)
        elif closest[1] == 8:
            return (closest[0], closest[1] + 1)


    def flashover(self):
        """ Convertir cualquier humo pegado a fuego en fuego mientras no haya puerta cerrada o pared."""
        for x in range(self.height):
            for y in range(self.width):

                if self.grid_status[x][y] == 1:
                    adjacent_cells = self.grid.get_neighborhood((x, y), moore = False, include_center = False)

                    index = 0
                    for position in adjacent_cells:
                        other_index = get_contrapart_cell_index(x, y, position[0], position[1])
                        if self.walls[x][y][other_index] == 0 and self.doors[x][y][other_index] != 2:
                            if self.grid_status[position[0]][position[1]] == 2:
                                self.grid_status[x][y] = 2
                                break
                        index += 1


    def check_poi_on_fire(self):
        """ Verifica si un punto de interés es afectado por el fuego. """
        for x in range(self.height):
            for y in range(self.width):

                if self.poi[x][y] != 0 and self.grid_status[x][y] == 2:
                    if self.poi[x][y] == 1:
                        choice = self._choose_between_victim_or_false_alarm(x, y)
                        if choice == 2:
                            self.lost_victims += 1

                    elif self.poi[x][y] == 2:
                        self.lost_victims += 1

                    self.poi[x][y] = 0


    def check_false_alarms(self):
        """ Quita la falsa alarma del tablero. """
        for x in range(self.height):
            for y in range(self.width):
                if self.poi[x][y] == 3:
                    self.poi[x][y] = 0


    def replenish_poi(self):
        """ Reponer los puntos de interés en las celdas vacías. """
        while self.point_of_interest < 3 and (self.total_victims > 0 or self.false_alarms > 0):
            poi_placed = False

            while not poi_placed:
                x, y = self.roll_dice()
                cell_poi = self.poi[x][y]
                cell_status = self.grid_status[x][y]

                if cell_poi == 0:
                    if cell_status != 0:
                        self.grid_status[x][y] = 0
                        self.poi[x][y] = 1

                    elif not self.grid.is_cell_empty((x, y)):
                        self.poi[x][y] = self._choose_between_victim_or_false_alarm(x, y)
                        
                    self.point_of_interest += 1
                    poi_placed = True
        
        self.game_phase = "Player"


    def _choose_between_victim_or_false_alarm(self, x, y):
        " Elige si el punto de interés es una víctima o una falsa alarma. "

        self.point_of_interest -= 1

        if self.poi_cells != []:
            for x_poi, y_poi, v_poi in self.poi_cells:
                if x_poi == x and y_poi == y:
                    self.poi_cells.remove((x_poi, y_poi, v_poi))
                    if v_poi == 2:
                        self.total_victims -= 1
                        return 2
                    elif v_poi == 3:
                        self.false_alarms -= 1
                        return 3
            

        if self.total_victims > 0 and self.false_alarms > 0:
            choice = random_number(2, 3)
            print("Choice de victima o alarma", choice)
            if choice == 2:
                self.total_victims -= 1
            elif choice == 3:
                self.false_alarms -= 1

        elif self.false_alarms == 0 and self.total_victims > 0:
            choice = 2
            self.total_victims -= 1

        elif self.total_victims == 0 and self.false_alarms > 0:
            choice = 3
            self.false_alarms -= 1
        else:
            choice = 0
            self.point_of_interest += 1

        return choice
        
        
    def roll_dice(self):
        """ Regresa un número aleatorio entre 1 y 8 y del 1 al 6. """
        return self.random.randint(1, 6), self.random.randint(1, 8)
    

    def is_game_over(self):
        """ Verifica si el juego ha terminado por victoria o derrota. """
        if self.damage_counters <= 0 or self.lost_victims >= 4:
            self.defeat = True
            return True
        elif self.saved_victims >= 7:
            self.victory = True
            return True
        else:
            return False
            
    def get_agents_info(self):
        agents_info = []
        for agent in self.schedule.agents:
            agents_info.append({
                "id": agent.unique_id,
                "position": {"x": agent.pos[0], "y": agent.pos[1]},
                "attributes": {  # Agrega otros datos relevantes del agente
                    "carrying_victim": agent.carrying_victim,
                    "knocked_down": agent.knocked_down,
                    "action_made": agent.action_made
                }
            })
        return agents_info

    def step(self):
        
        #print("Dentro de step modelo ", self.steps)
        #print(f"La fase actual es: {self.game_phase}")
        #print(f"El jugador activo es: {self.active_player}")
        #print(f"El contador de daño es: {self.damage_counters}")
        #print(f"El contador de falsas alarmas es: {self.false_alarms}")
        #print(f"El contador de victimcas perdidas es: {self.lost_victims}")
        #print(f"El contador de victimas totales es: {self.total_victims}")
        #print(f"El contador de puntos de interés es: {self.point_of_interest}")
        
        
        if self.game_phase == "Player":
            agent = self.agents_by_id[self.active_player]
            agent.step()
        elif self.game_phase == "Advance Fire":
            self.advance_fire()
        elif self.game_phase == "Secondary Effects":
            self.secondary_effects()
        elif self.game_phase == "Replenish POI":
            self.replenish_poi()

        self.datacollector.collect(self)
        self.steps += 1


### Simulación aleatoria

In [ ]:
walls_matrix, doors_matrix, fire_matrix, poi_matrix, poi_cells, outside_matrix = read_file("CasoPrueba1.txt")

model = FlashPointModel(walls_matrix, doors_matrix, fire_matrix, poi_matrix, poi_cells, outside_matrix)

while not model.is_game_over():
    model.step()

print("Steps: ", model.steps)


[[[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1]], [[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [1, 1, 0, 0], [0, 0, 0, 1]], [[0, 1, 1, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]], [[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 1], [1, 1, 0, 1]], [[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1]]]
[(2, 2), (2, 3), (3, 2), (3, 3), (3, 4), (3, 5), (4, 4), (5, 6), (5, 7), (6, 6)]
[(1, 3, 1, 4), (2, 5, 2, 6), (2, 8, 3, 8), (3, 2, 3, 3), (4, 4, 5, 4), (4, 6, 4, 7), (6, 5, 6, 6), (6, 7, 6, 8)]
[(1, 6), (3, 1), (4, 8), (6, 3)]
Puntos de acción 4
Posición actual (5, 0)
Posiciones posibles ((4, 0), (5, 1), (6, 0))
Í

## 2. Solución estratégica

### Funciones auxiliares

### Implementación del agente (estrategia)

In [ ]:
class FirefighterAgent_Strategy(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

        self.action_points = 4
        self.assigned_poi = None
        self.carrying_victim = False
        self.path_to_destination = []
        self.knocked_down = False
        self.points_saved = 0
        self.action_made = None

    
    def step(self):
        self.action_made = None

        if self.action_points > 0:
            # Checar si está cercano a un POI
            self._check_radius_for_poi()
            # Checar si está sobre un POI
            self._check_for_poi()

            # 1. Salvar víctima
            if self.carrying_victim:
                if self.path_to_destination == []:
                    self.path_to_destination = self.find_path_to_exterior()
                else:
                    turn_cost = self.pre_vision()   # NO considera costo de cargar víctima
                    self.take_action(turn_cost)
            
            # 2. Moverse a punto de interés
            elif self.assigned_poi is not None:
                if self.path_to_destination == []:
                    self.path_to_destination = self.find_path_to_poi()
                else:
                    turn_cost = self.pre_vision()
                    self.take_action(turn_cost)
            
            # 3. Apagar fuego
            else:
                if self.path_to_destination == []:
                    self.path_to_destination = self.find_path_to_fire()
                else:
                    turn_cost = self.pre_vision()
                    self.take_action(turn_cost)

        # Fin de turno
        elif self.action_points == 0:
            self.end_turn()


    def pre_vision(self):
        """ Previene las acciones que pueden realizarse con los puntos de acción de 
        ese turno y evita que el agente termine su turno adyascente a un fuego. """
        action_path = []
        path_followed = []
        pre_action_points = self.action_points
        curr_x, curr_y = self.pos

        # Acciones que pueden realizarse en ese turno
        for next_cell in self.path_to_destination:
            next_x, next_y = next_cell
            index = self._get_direction(curr_x, curr_y, next_x, next_y)

            adj_cells = self.model.grid.get_neighborhood((curr_x, curr_y), moore = False, include_center = False)
            for x, y in adj_cells:
                if self.model.grid_status[x][y] == 2 and pre_action_points >= 2:
                    action_path.append("extinguish")
                    pre_action_points -= 2
                    break
                elif self.model.grid_status[x][y] == 1 and pre_action_points >= 1:
                    action_path.append("extinguish")
                    pre_action_points -= 1
                    break


            if self.model.walls[curr_x][curr_y][index] == 1:
                if  pre_action_points >= 4:
                    action_path.append(("chop", 2))
                    action_path.append(("chop", 2))
                    pre_action_points -= 4
                elif pre_action_points >= 2:
                    action_path.append(("chop", 2))
                    pre_action_points -= 2
            
            elif self.model.walls[curr_x][curr_y][index] == 0.5 and pre_action_points >= 2:
                action_path.append(("chop", 2))
                pre_action_points -= 2
            
            elif self.model.doors[curr_x][curr_y][index] == 2 and pre_action_points >= 1:
                action_path.append(("toggle_door", 1))
                pre_action_points -= 1
            
            elif self.action_points >= 1:
                action_path.append(("move", 1))
                pre_action_points -= 1
            
            else:
                break

            curr_x, curr_y = next_x, next_y
            path_followed.append((curr_x, curr_y))

        # Verificar si el agente termina su turno adyascente a un fuego
        while self.model._fire_nearby_cost(curr_x, curr_y) != 0:
            last_action, last_cost = action_path.pop()
            last_pos = path_followed.pop()
            pre_action_points += last_cost

            if last_action == "move":
                curr_x, curr_y = last_pos
        
        return pre_action_points
    

    def take_action(self, turn_cost):
        """ Realiza las acciones que puede para seguir el camino al destino
        con los puntos de acción especificados. """

        curr_x, curr_y = self.pos

        # Acciones que pueden realizarse en ese turno
        for next_cell in self.path_to_destination:
            next_x, next_y = next_cell
            index_curr_to_adj = self._get_direction(curr_x, curr_y, next_x, next_y)
            index_adj_to_curr = get_contrapart_index(index_curr_to_adj)


            # Apagar fuegos adyacentes
            adj_cells = self.model.grid.get_neighborhood((curr_x, curr_y), moore = False, include_center = False)
            for x, y in adj_cells:
                if self.model.grid_status[x][y] == 2 and turn_cost >= 2:
                    turn_cost -= 2
                    self.action_points -= 2
                    self.model.grid_status[x][y] = 0
                    self.action_made = "extinguish fire"
                    return

                elif self.model.grid_status[x][y] == 1 and turn_cost >= 1:
                    turn_cost -= 1
                    self.action_points -= 1
                    self.model.grid_status[x][y] = 0
                    self.action_made = "extinguish smoke"
                    return
            

            # Dañar paredes sanas
            if self.model.walls[curr_x][curr_y][index_curr_to_adj] == 1 and turn_cost >= 2:
                turn_cost -= 2
                self.action_points -= 2
                self.model.walls[curr_x][curr_y][index_curr_to_adj] = 0.5
                self.model.walls[next_x][next_y][index_adj_to_curr] = 0.5
                self.model.damage_counters -= 1
                self.action_made = "chop"
                return
            
            # Dañar paredes dañadas
            elif self.model.walls[curr_x][curr_y][index_curr_to_adj] == 0.5 and turn_cost >= 2:
                turn_cost -= 2
                self.action_points -= 2
                self.model.walls[curr_x][curr_y][index_curr_to_adj] = 0
                self.model.walls[next_x][next_y][index_adj_to_curr] = 0
                self.model.damage_counters -= 1
                self.action_made = "chop"
                return
            
            # Abrir puertas
            elif self.model.doors[curr_x][curr_y][index_curr_to_adj] == 2 and turn_cost >= 1:
                turn_cost -= 1
                self.action_points -= 1
                self.model.doors[curr_x][curr_y][index_curr_to_adj] = 1
                self.model.doors[next_x][next_y][index_adj_to_curr] = 1
                self.action_made = "toggle_door"
                return
            
            # Moverse
            elif turn_cost >= 1:
                turn_cost -= 1
                self.action_points -= 1
                self.model.grid.move_agent(self, next_cell)
                self._check_for_poi(self.pos)
                self.action_made = "move"

                if self.carrying_victim:
                    self._save_victim()
                return
            
            # Si no se puede realizar ninguna acción, se termina el turno
            else:
                self.save_points()


    def find_path_to_exterior(self):
        """ Encuentra el camino más corto al exterior a través de las dos puertas
         más cercanas y la celda exterior más cercana a la posición actual. """
        x, y = self.pos
        door1, _, door2, _ = self.model.get_nearest_outside_doors(x, y)
        edge_door1 = self.model.get_adjacent_edge_cell(door1[0], door1[1])
        edge_door2 = self.model.get_adjacent_edge_cell(door2[0], door2[1])
        close_cell = self.model.get_closest_outside_cell(x, y)

        cost_door1, path_door1 = self._dijkstra(x, y, edge_door1[0], edge_door1[1])
        cost_door2, path_door2 = self._dijkstra(x, y, edge_door2[0], edge_door2[1])
        cost_cell, path_cell = self._dijkstra(x, y, close_cell[0], close_cell[1])

        chosen_cost = min(cost_door1, cost_door2, cost_cell)
        if chosen_cost == cost_door1:
            chosen_path = path_door1
        elif chosen_cost == cost_door2:
            chosen_path = path_door2
        else:
            chosen_path = path_cell
        return chosen_path
    

    def find_path_to_poi(self):
        """ Encuentra el camino menos costoso al punto de interés asignado. """
        x, y = self.pos
        x_poi, y_poi = self.assigned_poi
        path = self._dijkstra(x, y, x_poi, y_poi)
        return path
    

    def find_path_to_fire(self):
        """ Encuentra el camino menos costoso a la celda con fuego más cercana. """
        x, y = self.pos
        x_fire, y_fire = self.model.get_closest_fire(x, y)
        path = self._dijkstra(x, y, x_fire, y_fire)

        return path

  
    def toggle_door(self, position):
        """ Cambia el estado de la puerta en la celda (x, y). """
        
    
    
    def extinguish(self, position, option):
        """ Extingue el fuego o el humo en la celda (x, y). """
        

            

    def chop(self, index):
        """ Daña la pared en la celda (x, y). """
        



    def toggle_victim(self):
        """ Toma o deja a la víctima en la celda (x, y). """
        x, y = self.pos
        cell = self.model.poi[x][y]

        if cell == 2 and not self.carrying_victim:
            self.carrying_victim = True
            self.model.poi[x][y] = 0
            self.action_made = "carry_victim"

        elif cell == 0 and self.carrying_victim:
            self.carrying_victim = False
            self.model.poi[x][y] = 2
            self.action_made = "leave_victim"


    def save_points(self):
        self.points_saved = self.action_points
        self.action_made = "save_points"
        self.end_turn()


    def end_turn(self):
        """ Finaliza turno del agente, pasando a la siguiente fase de juego y al siguiente jugador. """
        self.action_points = 4 + self.points_saved
        self.points_saved = 0
        if self.action_points > 8:
            self.action_points = 8

        self.model.game_phase = "Advance Fire"
        self.action_made = "end_turn"

        if self.model.active_player < 5:
            self.model.active_player += 1
        else:
            self.model.active_player = 0


    # MÉTODOS AUXILIARES

    def _dijkstra(self, start, goal):
        """ Algoritmo de Dijkstra para encontrar el camino menos costoso entre dos celdas. """

        rows, cols = 8, 10                                      # Dimensiones del grid
        directions = [(-1, 0), (0, -1), (1, 0), (0, 1)]         # Arriba, Izquierda, Abajo, Derecha
        pq = []                                                 # Cola de prioridad
        costs = [[float('inf')] * cols for _ in range(rows)]    # Costos iniciales
        prev = [[None] * cols for _ in range(rows)]             # Rastreo de camino
        costs[start[0]][start[1]] = 0                           # Costo inicial
        heapq.heappush(pq, (0, start))                          # Nodo inicial en la cola

        while pq:
            current_cost, (x, y) = heapq.heappop(pq)            # Nodo con menor costo acumulado

            # Si se llega a la celda destino
            if (x, y) == goal:
                path = []
                while (x, y) is not None:
                    path.append((x, y))
                    x, y = prev[x][y]
                return current_cost, path[::-1]

            for i, (dx, dy) in enumerate(directions):
                nx, ny = x + dx, y + dy

                if 0 <= nx <= rows and 0 <= ny <= cols:
                    additional_cost = 1                     # Move
                    if self.model.walls[x][y][i] == 1 and self.model.damage_counters < 15:  # Pared              
                        additional_cost += 4
                    else:
                        continue

                    if self.model.walls[x][y][i] == 0.5 and self.model.damage_counters < 15:  # Pared dañada
                        additional_cost += 2
                    else:
                        continue

                    if self.model.doors[x][y][i] == 2:                  # Puerta cerrada
                        additional_cost += 1

                    if self.model.grid_status[nx][ny] == 2:            # Fuego en la siguiente celda
                        additional_cost += 2

                    additional_cost += self._fire_nearby_cost(nx, ny)   # Fuego en celdas adyacentes ??

                    new_cost = current_cost + additional_cost
                    if new_cost < costs[nx][ny]:            # Actualizar costo si es menor
                        costs[nx][ny] = new_cost
                        prev[nx][ny] = (x, y)
                        heapq.heappush(pq, (new_cost, (nx, ny)))

        return float('inf'), [] 
    

    def _fire_nearby_cost(self, position):
        """Verifica si hay fuego (valor 2 en grid_status) en las celdas alrededor de (nx, ny)."""
        neighbors = self.grid.get_neighborhood(position, moore=False, include_center=False)

        cost = 0
        for neighbor in neighbors:
            x, y = neighbor
            if self.grid_status[x][y] == 2:
                cost += 2
            elif self.grid_status[x][y] == 1:
                cost += 1

        return cost
    

    def _get_direction(current_x, current_y, next_x, next_y):
        """Determina la dirección de la celda next en relación a la celda current. """
        if next_x == current_x - 1 and next_y == current_y:
            return 0  # arriba
        elif next_x == current_x and next_y == current_y - 1:
            return 1  # izquierda
        elif next_x == current_x + 1 and next_y == current_y:
            return 2  # abajo
        elif next_x == current_x and next_y == current_y + 1:
            return 3  # derecha
        else:
            print("No son celdas adyacentes")
            return None
        

    def _check_radius_for_poi(self):
        """ Verifica si hay un punto de interés en las celdas adyacentes con un radio de 1."""
        neighbors = self.model.grid.get_neighborhood(self.pos, moore = True, include_center = True, radius = 1)

        for neighbor in neighbors:
            x, y = neighbor
            cell = self.model.poi[x][y]

            if (cell == 1 or cell == 2) and self.assigned_poi == []:
                # Si el POI no ha sido asignado
                if not any(coord == (x, y) for coord, _ in self.model.assigned_pois):
                    self.assigned_poi = (x, y)
                    self.model.assigned_pois.append((x, y))
                
                # Asignarlo al agente más cercano al POI (el que ya lo tenía vs el que lo acaba de encontrar)
                else:
                    for coord, agent in self.model.assigned_pois:
                        if coord == (x, y):
                            agent_cost, agent_path = self._dijkstra(agent.pos, coord)
                            self_cost, self_path = self._dijkstra(self.pos, coord)

                            min_cost = min(agent_cost, self_cost)
                            if min_cost == self_cost:
                                self.assigned_poi = (x, y)
                                self.model.assigned_pois.remove((coord, agent))
                                self.model.assigned_pois.append((x, y), self.unique_id)
                                self.path_to_destination = self_path
                                break
                            else:
                                break


    def _get_closest_fire(self):
        """ Obtiene la celda con fuego más cercana. """
        x, y = self.pos
        min_distance = float('inf')
        closest = None

        for x_fire, y_fire in self.model.fires:
            distance = abs(x - x_fire) + abs(y - y_fire)
            if distance < min_distance:
                min_distance = distance
                closest = (x_fire, y_fire)

        return closest
    

    def _check_for_poi(self, position):
        """ Verifica si la celda (x, y) tiene un punto de interés y qué tipo de punto es. """
        x, y = position
        cell = self.model.poi[x][y]

        if cell == 1:
            choice = self.model._choose_between_victim_or_false_alarm(x, y)
            self.model.poi[x][y] = choice
            cell = self.model.poi[x][y]

        if cell == 2:
            self.carrying_victim = True
            self.model.poi[x][y] = 0


    def _save_victim(self):
        """ Deja a la víctima en una celda exterior. """
        if self.carrying_victim and self.pos in self.model.edge_positions:
            self.carrying_victim = False
            self.model.victims_saved += 1
        
        
    def _get_cell_status(self, position):
        """ Obtiene el contenido de una celda (humo o fuego). """
        x, y = position
        return self.model.grid_status[x][y]
    

### Implementación del modelo (estrategia)

In [ ]:
class FlashPointModel_Strategy(Model):

    def __init__(self, walls_matrix, doors_matrix, fire_matrix, poi_matrix, poi_cells, outside_matrix, 
                 width = 10, height = 8, agents = 6, damage_counters = 24):

        super().__init__()
        self.grid = MultiGrid(height, width, torus = False)
        self.schedule = SimultaneousActivation(self)

        self.datacollector = DataCollector()

        # Matrices
        self.walls = walls_matrix
        self.doors = doors_matrix
        self.outside_doors = outside_matrix
        self.grid_status = fire_matrix
        self.poi = poi_matrix
        self.poi_cells = poi_cells

        self.saved_victims = 0
        self.lost_victims = 0
        self.damage_counters = damage_counters
        self.false_alarms = 6
        self.total_victims = 12
        self.point_of_interest = 3
        self.steps = 0
        self.width = width
        self.height = height
        self.game_phase = "Player"
        self.agents_by_id = {}
        self.active_player = 0
        self.victory = False
        self.defeat = False
        self.assigned_pois = []

        self.edge_positions = self.get_edge_positions(width, height)

        # Creación agentes
        for i in range(agents):
            agent = FirefighterAgent_Strategy(i, self)
            self.schedule.add(agent)
            self.agents_by_id[i] = agent

            agent_placed = False
            while not agent_placed:
                
                # Posicionar cerca de POIs
                if len(self.assigned_pois) < 3:
                    for x, y, _ in self.poi_cells:
                        if (x, y) not in self.assigned_pois:
                            near_edge, dist1 = self.get_nearest_edge_cell(x, y)
                            near_door1, dist2, near_door2, dist3 = self.get_nearest_outside_doors(x, y)

                            # Obtener dist mínimo
                            min_dist = min(dist1, dist2, dist3)
                            if min_dist == dist1:
                                min_pos = near_edge
                            elif min_dist == dist2:
                                min_pos = near_door1
                            else:
                                min_pos = near_door2

                            if self.grid.is_cell_empty(min_pos):
                                self.grid.place_agent(agent, min_pos)
                                agent_placed = True
                                agent.assigned_poi = (x, y)        
                                self.assigned_pois.append((x, y), agent.unique_id)
                                break

                # Posicionar en puerta exterior aleatoria
                else:
                    chosen_door = random.choice(self.outside_doors)
                    chosen_pos = self.get_adjacent_edge_cell(chosen_door[0], chosen_door[1])
                    if self.grid.is_cell_empty(chosen_pos):
                        self.grid.place_agent(agent, chosen_pos)
                        agent_placed = True


    def get_edge_positions(self, width, height):
        """ Obtiene las celdas de los bordes del tablero. """
        edge_positions = []
        for x in range(0, height - 1):
            edge_positions.append((x, 0))  # Orilla superior
            edge_positions.append((x, 9))  # Orilla inferior

        for y in range(0, width - 1):
            edge_positions.append((0, y))  # Orilla izquierda
            edge_positions.append((7, y))  # Orilla derecha

        return edge_positions
    

    def get_adjacent_edge_cell(self, x, y):
        """ Obtiene la celda del exterior adyacente a la celda (x, y). """
        if x == 1:
            return (0, y)
        elif x == 6:
            return (7, y)
        elif y == 1:
            return (x, 0)
        elif y == 8:
            return (x, 9)
        

    def get_nearest_edge_cell(self, x, y):
        """ Obtiene la celda del exterior más cercana a la celda (x, y). """
        closest = None

        if x <= 3:
            closest = (0, y)
        elif x <= 6:
            closest = (7, y)
        elif y <= 4:
            closest = (x, 0)
        elif y <= 8:
            closest = (x, 9)

        distance = abs(x - closest[0]) + abs(y - closest[1])
        return closest, distance
    

    def get_nearest_outside_doors(self, x, y):
        """ Obtiene las dos puertas exteriores más cercanas a la celda (x, y). """
        distances = []

        for door in self.outside_doors:
            distance = abs(x - door[0]) + abs(y - door[1])
            distances.append((distance, door))

        # Ordenar las puertas por distancia
        distances.sort(key=lambda d: d[0])

                # Primer puerta     Dist       Segunda puerta       Dist
        return distances[0][1], distances[0][0], distances[1][1], distances[1][0]
    

    def advance_fire(self):
        """ Coloca un humo aletorio. """
        x, y = self.roll_dice()
        cell = self.grid_status[x][y]
        
        # Si la celda está vacía, se coloca humo
        if cell == 0:
            self.grid_status[x][y] = 1
        elif cell == 1:
            self.grid_status[x][y] = 2
        elif cell == 2:
            self.explosion(x, y)
        
        self.game_phase = "Secondary Effects"


    def explosion(self, x, y):
        """ Realiza una explosión y llama a explosion_effect para las celdas adyacentes. """
        print("Dentro de explosion")
        adjacent_cells = self.grid.get_neighborhood((x, y), moore = False, include_center = False)
        
        index = 0
        for position in adjacent_cells:
            self.explosion_effect(position[0], position[1], index, x, y)
            index += 1
        
        
    def explosion_effect(self, adj_x, adj_y, index, current_x, current_y):
        """ Verifica qué encuentra en las celdas adyacentes y genera el efecto correspondiente. """
        # Index 0 arriba, 1 izquierda, 2 derecha, 3 abajo
        cell = self.grid_status[adj_x][adj_y]
        index_curr_to_adj = get_contrapart_cell_index(current_x, current_y, adj_x, adj_y)
        index_adj_to_curr = get_contrapart_index(index_curr_to_adj)
        door = self.doors[current_x][current_y][index_curr_to_adj]
        wall = self.walls[current_x][current_y][index_curr_to_adj]
        
        # Puerta cerrada se destruye
        if door == 2:
            self.doors[adj_x][adj_y][index_adj_to_curr] = 0
            self.doors[current_x][current_y][index_curr_to_adj] = 0  
            print("Puerta destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
            print("Puerta destruida en la celda", adj_x, adj_y, "de lado", index_adj_to_curr)

        # Puerta abierta se destruye y avanza fuego
        elif door == 1:
            self.doors[adj_x][adj_y][index_adj_to_curr] = 0
            self.doors[current_x][current_y][index_curr_to_adj] = 0
            self.grid_status[adj_x][adj_y] = 2
            print("Puerta destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
            print("Puerta destruida en la celda", adj_x, adj_y, "de lado", index_adj_to_curr)

        # Pared se daña
        elif wall == 1 or wall == 0.5:
            self.walls[adj_x][adj_y][index_adj_to_curr] -= 0.5
            self.walls[current_x][current_y][index_curr_to_adj] -= 0.5
            self.damage_counters -= 1
            print("Pared dañada en la celda", current_x, current_y, "de lado", index_curr_to_adj)
            print("Pared dañada en la celda", adj_x, adj_y, "de lado", index_adj_to_curr)

        elif wall == 0:
            if cell == 2:
                print("Shockwave en la celda", adj_x, adj_y, "en dirección", index_curr_to_adj)
                self.shockwave(adj_x, adj_y, index_curr_to_adj)

            # Prende fuego a la siguiente celda
            else:
                self.grid_status[adj_x][adj_y] = 2
                print("Fuego en la celda", adj_x, adj_y)

                
                
    def shockwave(self, initial_x, initial_y, index_curr_to_adj):
        """ Realiza una onda expansiva de llammas en una dirección hasta no encontrar más fuego."""
        print("Dentro de shockwave")
        keep_going = True
        current_x, current_y = initial_x, initial_y

        while keep_going:
            adjacent_cells = self.grid.get_neighborhood((current_x, current_y), moore = False, include_center = False)
            # La siguiente celda es siempre en la misma dirección
            next_fire = adjacent_cells[index_curr_to_adj]
            next_x, next_y = next_fire

            # Shockwave avanza a través del fuego SI NO HAY PUERTA O PARED
            door = self.doors[current_x][current_y][index_curr_to_adj]
            wall = self.walls[current_x][current_y][index_curr_to_adj]
            index_adj_to_curr = get_contrapart_index(index_curr_to_adj)

            if self.grid_status[next_x][next_y] == 2:
                if door != 2 and wall != 1:
                    # Puerta abierta se destruye y avanza fuego
                    if door == 1:
                        self.doors[next_x][next_y][index_adj_to_curr] = 0
                        self.doors[current_x][current_y][index_curr_to_adj] = 0
                        self.grid_status[next_x][next_y] = 2
                        print("Puerta destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
                        print("Puerta destruida en la celda", next_x, next_y, "de lado", index_adj_to_curr)

                    # Pared dañada se destruye y avanza fuego
                    elif wall == 0.5:
                        self.walls[next_x][next_y][index_adj_to_curr] == 0
                        self.walls[current_x][current_y][index_curr_to_adj] == 0
                        self.grid_status[next_x][next_y] = 2
                        self.damage_counters -= 1
                        print("Pared destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
                        print("Pared destruida en la celda", next_x, next_y, "de lado", index_adj_to_curr)

                    print("Fuego avanza a la celda", next_x, next_y)
                    current_x, current_y = next_fire

            else:
                keep_going = False  
                print("Se rompe shockwave en la celda", next_x, next_y)
                
                # Puerta cerrada se destruye
                if door == 2:
                    self.doors[next_x][next_y][index_adj_to_curr] = 0
                    self.doors[current_x][current_y][index_curr_to_adj] = 0 
                    print("Puerta destruida en la celda", current_x, current_y, "de lado", index_curr_to_adj)
                    print("Puerta destruida en la celda", next_x, next_y, "de lado", index_adj_to_curr) 

                # Pared se daña
                elif wall == 1:
                    self.walls[next_x][next_y][index_adj_to_curr] -= 0.5
                    self.walls[current_x][current_y][index_curr_to_adj] -= 0.5
                    self.damage_counters -= 1
                    print("Pared dañada en la celda", current_x, current_y, "de lado", index_curr_to_adj)
                    print("Pared dañada en la celda", next_x, next_y, "de lado", index_adj_to_curr)

                # Se coloca fuego continuo
                else:
                    self.grid_status[next_x][next_y] = 2
                    print("Fuego en la celda", next_x, next_y)

    def secondary_effects(self):
        """ Verifica los efectos secundarios tras la aparición de humo o fuego. """
        self.clean_fire_on_edges()
        self.flashover()
        self.knock_down()
        self.check_poi_on_fire()
        self.game_phase = "Replenish POI"
        

    def clean_fire_on_edges(self):
        """ Limpia el fuego en las celdas de los bordes. """
        for x, y in self.edge_positions:
            self.grid_status[x][y] = 0



    def knock_down(self):
        """ Verifica si un bombero es derribado por el fuego y lo coloca en la puerta más cercana. """
        for agent in self.schedule.agents:
            x, y = agent.pos
            cell = self.grid_status[x][y]
            if cell == 2:
                if agent.carrying_victim:
                    self.lost_victims += 1
                    agent.carrying_victim = False
                agent.knocked_down = True

                # Colocarlo en la puerta externa más cercana
                self.grid.move_agent(agent, self.get_closest_door(x, y))


    def get_closest_door(self, x, y):
        """ Obtiene la puerta externa más cercana a la celda (x, y). """
        min_distance = float('inf')
        closest = None

        for door in self.outside_doors:
            distance = abs(x - door[0]) + abs(y - door[1])
            if distance < min_distance:
                min_distance = distance
                closest = door

        if closest[0] == 1:
            return (closest[0] - 1, closest[1])
        elif closest[0] == 6:
            return (closest[0] + 1, closest[1])
        elif closest[1] == 1:
            return (closest[0], closest[1] - 1)
        elif closest[1] == 8:
            return (closest[0], closest[1] + 1)


    def flashover(self):
        """ Convertir cualquier humo pegado a fuego en fuego."""
        for x in range(self.height):
            for y in range(self.width):

                if self.grid_status[x][y] == 1:
                    adjacent_cells = self.grid.get_neighborhood((x, y), moore = False, include_center = False)

                    for position in adjacent_cells:
                        if self.grid_status[position[0]][position[1]] == 2:
                            self.grid_status[x][y] = 2
                            break


    def check_poi_on_fire(self):
        """ Verifica si un punto de interés es afectado por el fuego. """
        for x in range(self.height):
            for y in range(self.width):

                if self.poi[x][y] != 0 and self.grid_status[x][y] == 2:
                    if self.poi[x][y] == 1:
                        choice = self._choose_between_victim_or_false_alarm(x, y)
                        if choice == 2:
                            self.lost_victims += 1

                    elif self.poi[x][y] == 2:
                        self.lost_victims += 1

                    self.poi[x][y] = 0


    def replenish_poi(self):
        """ Reponer los puntos de interés en las celdas vacías. """
        while self.point_of_interest < 3 and (self.total_victims > 0 or self.false_alarms > 0):
            poi_placed = False

            while not poi_placed:
                x, y = self.roll_dice()
                cell_poi = self.poi[x][y]
                cell_status = self.grid_status[x][y]

                if cell_poi == 0:
                    if cell_status != 0:
                        self.grid_status[x][y] = 0
                        self.poi[x][y] = 1

                    elif not self.grid.is_cell_empty((x, y)):
                        self.poi[x][y] = self._choose_between_victim_or_false_alarm(x, y)
                        
                    self.point_of_interest += 1
                    poi_placed = True
        
        self.game_phase = "Player"


    def _choose_between_victim_or_false_alarm(self, x, y):
        " Elige si el punto de interés es una víctima o una falsa alarma. "

        self.point_of_interest -= 1

        if self.poi_cells != []:
            for x_poi, y_poi, v_poi in self.poi_cells:
                if x_poi == x and y_poi == y:
                    self.poi_cells.remove((x_poi, y_poi, v_poi))
                    if v_poi == 2:
                        self.total_victims -= 1
                        return 2
                    elif v_poi == 3:
                        self.false_alarms -= 1
                        return 3
            

        if self.total_victims > 0 and self.false_alarms > 0:
            choice = random_number(2, 3)
            print("Choice de victima o alarma", choice)
            if choice == 2:
                self.total_victims -= 1
            elif choice == 3:
                self.false_alarms -= 1

        elif self.false_alarms == 0 and self.total_victims > 0:
            choice = 2
            self.total_victims -= 1

        elif self.total_victims == 0 and self.false_alarms > 0:
            choice = 3
            self.false_alarms -= 1
        else:
            choice = 0
            self.point_of_interest += 1

        return choice
        
        
    def roll_dice(self):
        """ Regresa un número aleatorio entre 1 y 8 y del 1 al 6. """
        return self.random.randint(1, 6), self.random.randint(1, 8)
    

    def is_game_over(self):
        """ Verifica si el juego ha terminado por victoria o derrota. """
        if self.damage_counters <= 0 or self.lost_victims >= 4:
            self.defeat = True
            return True
        elif self.saved_victims >= 7:
            self.victory = True
            return True
        else:
            return False
            
    def get_agents_info(self):
        agents_info = []
        for agent in self.schedule.agents:
            agents_info.append({
                "id": agent.unique_id,
                "position": {"x": agent.pos[0], "y": agent.pos[1]},
                "attributes": {  # Agrega otros datos relevantes del agente
                    "carrying_victim": agent.carrying_victim,
                    "knocked_down": agent.knocked_down,
                }
            })
        return agents_info

    def step(self):
        
        #print("Dentro de step modelo ", self.steps)
        #print(f"La fase actual es: {self.game_phase}")
        #print(f"El jugador activo es: {self.active_player}")
        #print(f"El contador de daño es: {self.damage_counters}")
        #print(f"El contador de falsas alarmas es: {self.false_alarms}")
        #print(f"El contador de victimcas perdidas es: {self.lost_victims}")
        #print(f"El contador de victimas totales es: {self.total_victims}")
        #print(f"El contador de puntos de interés es: {self.point_of_interest}")
        
        
        if self.game_phase == "Player":
            agent = self.agents_by_id[self.active_player]
            agent.step()
        elif self.game_phase == "Advance Fire":
            self.advance_fire()
        elif self.game_phase == "Secondary Effects":
            self.secondary_effects()
        elif self.game_phase == "Replenish POI":
            self.replenish_poi()

        self.datacollector.collect(self)
        self.steps += 1


### Simulación estratégica

In [ ]:
walls_matrix, doors_matrix, fire_matrix, poi_matrix, poi_cells, outside_matrix = read_file("CasoPrueba1.txt")

model = FlashPointModel_Strategy(walls_matrix, doors_matrix, fire_matrix, poi_matrix, poi_cells, outside_matrix)

while not model.is_game_over():
    model.step()

print("Steps: ", model.steps)

## Conexión con el cliente

In [ ]:
# TC2008B Modelación de Sistemas Multiagentes con gráficas computacionales
# Python server to interact with Unity via POST
# Sergio Ruiz-Loza, Ph.D. March 2021

from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import json

# Crear una matriz 8x10 para el piso
matrix = create_border_matrix(8, 10)

walls_matrix, doors_matrix, fire_matrix, poi_matrix, poi_cells, outside_matrix = read_file("CasoPrueba1.txt")
model = FlashPointModel(walls_matrix, doors_matrix, fire_matrix, poi_matrix, poi_cells, outside_matrix)

class Server(BaseHTTPRequestHandler):
    
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
    def do_GET(self):
        self._set_response()
        self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    def do_POST(self):
        response = {
            "steps": model.steps,
            "game_phase": model.game_phase,
            "active_player": model.active_player,
            "damage_counters": model.damage_counters,
            "false_alarms": model.false_alarms,
            "lost_victims": model.lost_victims,
            "total_victims": model.total_victims,
            "point_of_interest": model.point_of_interest,
            "grid_status": model.grid_status.tolist(),
            "walls": model.walls,
            "floor": matrix,
            "doors": model.doors,
            "poi": model.poi.tolist(),
            "agents": model.get_agents_info(),
            "active_agent": model.active_player
        }
        model.step()

        self._set_response()
        self.wfile.write(json.dumps(response).encode('utf-8'))


def run(server_class=HTTPServer, handler_class=Server, port=8585):
    logging.basicConfig(level=logging.INFO)
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    logging.info("Starting httpd...\n") # HTTPD is HTTP Daemon!
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:   # CTRL+C stops the server
        pass
    httpd.server_close()
    logging.info("Stopping httpd...\n")

if __name__ == '__main__':
    from sys import argv
    
    if len(argv) == 2:
        run(port=int(argv[1]))
    else:
        run()

INFO:root:Starting httpd...



[[[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1]], [[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [1, 1, 0, 0], [0, 0, 0, 1]], [[0, 1, 1, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]], [[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 1], [1, 1, 0, 1]], [[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1]]]
[(2, 2), (2, 3), (3, 2), (3, 3), (3, 4), (3, 5), (4, 4), (5, 6), (5, 7), (6, 6)]
[(1, 3, 1, 4), (2, 5, 2, 6), (2, 8, 3, 8), (3, 2, 3, 3), (4, 4, 5, 4), (4, 6, 4, 7), (6, 5, 6, 6), (6, 7, 6, 8)]
[(1, 6), (3, 1), (4, 8), (6, 3)]
Puntos de acción 4
Acción realizada move 



127.0.0.1 - - [25/Nov/2024 12:11:49] "POST / HTTP/1.1" 200 -


Puntos de acción 3
Acción realizada save_points 



127.0.0.1 - - [25/Nov/2024 12:11:55] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:14:16] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:14:31] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:14:38] "POST / HTTP/1.1" 200 -


...

127.0.0.1 - - [25/Nov/2024 12:15:49] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:15:50] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:15:51] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:15:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:15:54] "POST / HTTP/1.1" 200 -


Puntos de acción 4
Acción realizada move 



127.0.0.1 - - [25/Nov/2024 12:15:54] "POST / HTTP/1.1" 200 -


Puntos de acción 3
Acción realizada save_points 



127.0.0.1 - - [25/Nov/2024 12:15:55] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:15:55] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 12:15:58] "POST / HTTP/1.1" 200 -
